In [3]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [4]:
train_trans = pd.read_csv('data/train_trans.csv')

In [5]:
train_base = pd.read_csv('data/train_base.csv')
test_a_base = pd.read_csv('data/test_a_base.csv')

train_op = pd.read_csv('data/train_op.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')

train_trans = pd.read_csv('data/train_trans.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

train_label = pd.read_csv('data/train_label.csv')

In [6]:
train_base = pd.merge(train_base, train_label, on='user', how='left')

## base处理

In [123]:
train_base_no_nan = train_base.drop(columns=['service3_level'],axis=1)
# 离散值填充众数
train_base_no_nan['sex'].fillna('category 0',inplace=True)
train_base_no_nan['balance_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance1_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance2_avg'].fillna('level 1',inplace=True)

In [124]:
user = train_base_no_nan['user'].values
train_base_no_user = train_base_no_nan.drop(columns=['user'],axis=1)

In [125]:
category_columns_name = ['sex','provider','level','verified','agreement1','agreement2','agreement3','agreement4']

one_hot_df = pd.get_dummies(train_base_no_user[category_columns_name])

In [126]:
train_base_categoty = train_base_no_user.select_dtypes('O')

for c in list(train_base_categoty.columns[0:9])+['service3']:
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))
    
for c in list(train_base_categoty.columns[11:17])+list(train_base_categoty.columns[18:25]):
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))

In [12]:
def mean_coding(data, feature, target='label'):
    n_folds = 10
    n_inner_folds = 5
    mean_coded = pd.Series()

    # 所有数据的label均值
    default_mean = data[target].mean()
    kf = KFold(n_splits=n_folds, shuffle=True)

    out_mean_cv = pd.DataFrame()
    split = 0
    # 对所有数据做CV
    for in_fold, out_fold in tqdm(kf.split(data[feature])):

        impact_coded_cv = pd.Series()
        kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
        inner_split = 0
        inner_mean_cv = pd.DataFrame()

        # in_fold数据的label均值
        default_inner_mean = data.iloc[in_fold][target].mean()

        # 对in_fold数据做CV
        for in_fold_inner, out_fold_inner in kf_inner.split(data.iloc[in_fold]):
            # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)

            # 对in_fold_inner做group_by求出mean:feature-mean
            in_fold_inner_mean = data.iloc[in_fold_inner].groupby(by=feature)[target].mean()

            # in_fold的mean使用cv后的in_fold_inner的mean值，如果没有在里面，就用in_fold的label均值
            impact_coded_cv = impact_coded_cv.append(data.iloc[in_fold].apply(
                lambda x: in_fold_inner_mean[x[feature]]
                if x[feature] in in_fold_inner_mean.index
                else default_inner_mean
                , axis=1))

            # Also populate mapping (this has all group -> mean for all inner CV folds)
            inner_mean_cv = inner_mean_cv.join(pd.DataFrame(in_fold_inner_mean), rsuffix=inner_split, how='outer')
            inner_mean_cv.fillna(value=default_inner_mean, inplace=True)
            inner_split += 1

        # Also populate mapping
        out_mean_cv = out_mean_cv.join(pd.DataFrame(inner_mean_cv), rsuffix=split, how='outer')
        out_mean_cv.fillna(value=default_mean, inplace=True)
        split += 1

        mean_coded = mean_coded.append(data.iloc[out_fold].apply(
            lambda x: inner_mean_cv.loc[x[feature]].mean()
            if x[feature] in inner_mean_cv.index
            else default_mean
            , axis=1))

    return mean_coded, out_mean_cv.mean(axis=1), default_mean

In [13]:
city_mean, city_mapping, city_default_means = mean_coding(train_base_no_user, 'city')

10it [01:17,  7.83s/it]


In [14]:
province_mean, province_mapping, province_default_means = mean_coding(train_base_no_user, 'province')

10it [01:20,  8.00s/it]


In [15]:
regist_type_mean, regist_type_mapping, regist_type_default_means = mean_coding(train_base_no_user, 'regist_type')

10it [01:18,  7.92s/it]


In [103]:
balance_mean, balance_mapping, balance_default_means = mean_coding(train_base_no_user, 'balance')

10it [01:19,  8.22s/it]


In [104]:
balance_avg_mean, balance_avg_mapping, balance_avg_default_means = mean_coding(train_base_no_user, 'balance_avg')

10it [01:22,  8.06s/it]


In [105]:
balance1_mean, balance1_mapping, balance1_default_means = mean_coding(train_base_no_user, 'balance1')

10it [01:16,  7.59s/it]


In [106]:
balance1_avg_mean, balance1_avg_mapping, balance1_avg_default_means = mean_coding(train_base_no_user, 'balance1_avg')

10it [01:15,  7.48s/it]


In [107]:
balance2_mean, balance2_mapping, balance2_default_means = mean_coding(train_base_no_user, 'balance2')

10it [01:13,  7.34s/it]


In [108]:
balance2_avg_mean, balance2_avg_mapping, balance2_avg_default_means = mean_coding(train_base_no_user, 'balance2_avg')

10it [01:16,  7.70s/it]


In [16]:
joblib.dump(city_mean,'city_mean')
joblib.dump(province_mean,'province_mean')
joblib.dump(regist_type_mean,'regist_type_mean')

['regist_type_mean']

In [127]:
train_base_categoty['city'] = city_mean
train_base_categoty['province'] = province_mean
train_base_categoty['regist_type'] = regist_type_mean
train_base_categoty['balance'] = balance_mean
train_base_categoty['balance_avg'] = balance_avg_mean
train_base_categoty['balance1'] = balance1_mean
train_base_categoty['balance1_avg'] = balance1_avg_mean
train_base_categoty['balance2'] = balance2_mean
train_base_categoty['balance2_avg'] = balance2_avg_mean

In [ ]:
# cumsu = train_base_no_user.groupby('city')['label'].cumsum() - train_base_no_user['label']
# cumcnt = train_base_no_user.groupby('city').cumcount()+1e-10
# train_base_categoty['city'] = cumsu/cumcnt

# cumsu = train_base_no_user.groupby('province')['label'].cumsum() - train_base_no_user['label']
# cumcnt = train_base_no_user.groupby('province').cumcount()+1e-10
# train_base_categoty['province'] = cumsu/cumcnt

In [ ]:
# city_dic = dict(train_base_no_user.groupby('city')['label'].sum()/train_base_no_user.groupby('city')['label'].count())
# province_dic = dict(train_base_no_user.groupby('province')['label'].sum()/train_base_no_user.groupby('province')['label'].count())

# train_base_categoty['city'] = train_base_categoty['city'].apply(lambda x:city_dic[x])
# train_base_categoty['province'] = train_base_categoty['province'].apply(lambda x:province_dic[x])

In [ ]:
# joblib.dump(city_dic,'city_dic.pkl')
# joblib.dump(province_dic,'province_dic.pkl')

In [128]:
# train_base_categoty.drop(columns=['province','city'],axis=1,inplace=True)
train_base_categoty.drop(columns=category_columns_name,axis=1,inplace=True)

In [117]:
# train_base_categoty.drop(columns=['balance','balance_avg','balance1','balance1_avg','balance2','balance2_avg'],axis=1,inplace=True)

In [129]:
number_value = train_base_no_user.select_dtypes('int64').values
df_value= pd.DataFrame(number_value)
df_value.columns = train_base_no_user.select_dtypes('int64').columns

In [130]:
df_value['card_cnt'] = df_value['card_a_cnt']+df_value['card_b_cnt']+df_value['card_c_cnt']+df_value['card_d_cnt']
df_value['ip_cnt_avg'] = df_value['ip_cnt']/df_value['login_days_cnt']

df_value['login_cnt_period']=df_value['login_cnt_period1']+df_value['login_cnt_period2']
df_value['login_cnt_period_avg']=df_value['login_cnt_period']/df_value['login_days_cnt']
df_value['login_cnt_period1_avg']=df_value['login_cnt_period1']/df_value['login_days_cnt']
df_value['login_cnt_period2_avg']=df_value['login_cnt_period2']/df_value['login_days_cnt']

df_value['using_time_avg'] = df_value['using_time']/df_value['login_days_cnt']

df_value['product7_success_cnt'] = df_value['product7_cnt']-df_value['product7_fail_cnt']
df_value['product7_success_cnt_avg'] = df_value['product7_success_cnt']/df_value['login_days_cnt']
df_value['product7_cnt_avg']=df_value['product7_cnt']/df_value['login_days_cnt']
df_value['product7_fail_cnt_avg']=df_value['product7_fail_cnt']/df_value['login_days_cnt']

df_value['service_cnt']=df_value['service1_cnt']+df_value['service2_cnt']
df_value['service_cnt_avg']= df_value['service_cnt']/df_value['login_days_cnt']
df_value['service_amt_cnt_avg']=df_value['service1_amt']/df_value['service_cnt']

In [131]:
# 归一化
df_value = (df_value-df_value.min())/(df_value.max()-df_value.min())
# df_value = (df_value-df_value.mean())/df_value.std()

In [132]:
df_base = pd.DataFrame()
df_base['user'] = user

df_base = pd.concat([df_base, train_base_categoty,df_value,one_hot_df],axis=1)

In [150]:
df_base['service_cnt'].value_counts()

0.000000    46892
0.109091      796
0.054545       50
0.236364       27
0.172727        7
0.118182        2
0.345455        2
0.454545        2
0.227273        2
0.563636        1
1.000000        1
Name: service_cnt, dtype: int64

In [133]:
df_base.head()

,user,regist_type,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,label,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period_avg,login_cnt_period1_avg,login_cnt_period2_avg,using_time_avg,product7_success_cnt,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,sex_category 0,sex_category 1,provider_category 0,provider_category 1,provider_category 2,level_category 0,level_category 1,level_category 2,verified_category 0,verified_category 1,agreement1_category 0,agreement1_category 1,agreement2_category 0,agreement2_category 1,agreement3_category 0,agreement3_category 1,agreement4_category 0,agreement4_category 1
0,Train_06800,0.273967,0.230296,0.203812,0.286672,0.348703,0.281810,0.297835,0.269883,0.332909,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.0,0.057692,0.225496,0.003810,0.003509,0.004135,0.002801,0.883054,0.000000,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,1,0
1,Train_23487,0.273964,0.197882,0.204529,0.217926,0.226309,0.281810,0.297839,0.213411,0.225405,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,1.0,0.125000,0.235239,0.004431,0.003695,0.004076,0.003265,0.783694,0.857143,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0,1,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1
2,Train_36880,0.273967,0.231331,0.257419,0.230183,0.258607,0.184612,0.204396,0.269890,0.225405,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,1.0,0.125000,0.270565,0.014887,0.013557,0.017896,0.008655,0.645769,0.857143,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0
3,Train_35392,0.249028,0.201054,0.185553,0.260362,0.245544,0.281810,0.297848,0.269553,0.197110,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.0,0.057692,0.243241,0.003550,0.002860,0.002696,0.003046,0.775081,0.857143,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0
4,Train_35057,0.310016,0.316468,0.263675,0.205005,0.221704,0.281807,0.297836,0.204594,0.225404,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,1.0,0.120192,0.225519,0.002607,0.001898,0.003001,0.000651,0.811092,0.857143,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1


## OP处理

In [23]:
op_df = pd.DataFrame()
group = train_op.groupby(['user']).count()
op_df['user'] = group.index

In [24]:
count_column=train_op.columns[1:-1]
for column in count_column:
    print(column)
#     des_df = train_op.groupby('user')[column].describe()
#     op_df['op_'+column+'_count'] = des_df['count'].values
#     op_df['op_'+column+'_n'] = des_df['unique'].values
#     op_df['op_'+column+'_fre'] = des_df['freq'].values
    
    op_df['op_'+column+'_count'] = train_op.groupby('user')[column].count().values
    op_df['op_'+column+'_count'] = train_op.groupby('user')[column].nunique().values
    

op_type
op_mode
op_device
ip
net_type
channel
ip_3


In [25]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

train_op['time_diff_sec'] = train_op['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    op_df['op_time_'+operate]=train_op.groupby('user')['time_diff_sec'].agg(operate).values

In [26]:
op_df.fillna(0,inplace=True)

In [27]:
op_df.head()

,user,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count,op_net_type_count,op_channel_count,op_ip_3_count,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std
0,Train_00000,11,12,2,3,3,4,3,683286.0,219874.0,3.058014e+05,263361.0,125051.535485
1,Train_00001,3,3,1,2,2,3,2,1263768.0,409809.0,6.598048e+05,438553.5,385614.886428
2,Train_00002,3,3,1,1,2,3,1,1283043.0,1282741.0,1.282845e+06,1282751.5,139.053690
3,Train_00003,8,8,2,18,2,3,5,1278388.0,199014.0,9.408690e+05,957412.0,247893.797669
4,Train_00004,2,2,1,1,1,2,1,631021.0,630973.0,6.309866e+05,630975.0,20.659138


In [55]:
op_df.shape

(41892, 13)

## 交易处理

In [28]:
trans_df = pd.DataFrame()
group = train_trans.groupby(['user']).count()
trans_df['user'] = group.index

### 离散列，全部统计有几个类别

In [29]:
count_column = list(train_trans.columns[1:4])+list(train_trans.columns[5:9]) 
for column in count_column:
    print(column)
    trans_df['trans_'+column+'_count'] = train_trans.groupby('user')[column].count().values
    trans_df['trans_'+column+'_count'] = train_trans.groupby('user')[column].nunique().values
#     des_df = train_trans.groupby('user')[column].describe()
#     trans_df['trans_'+column+'_count'] = des_df['count'].values
#     trans_df['trans_'+column+'_n'] = des_df['unique'].values
#     trans_df['trans_'+column+'_fre'] = des_df['freq'].values

platform
tunnel_in
tunnel_out
type1
ip
type2
ip_3


### 处理amount

In [30]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=train_trans.groupby(['user'])['amount'].agg(operate).values

### 处理时间

In [31]:
train_trans['time_diff_sec'] = train_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=train_trans.groupby('user')['time_diff_sec'].agg(operate).values

### 标准差、freq有空值，用0填充

In [32]:
trans_df.fillna(0,inplace=True)

In [33]:
trans_df.head()

,user,trans_platform_count,trans_tunnel_in_count,trans_tunnel_out_count,trans_type1_count,trans_ip_count,trans_type2_count,trans_ip_3_count,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_00000,2,1,1,4,2,1,2,267542,24798,53330.307692,30746.0,65274.138488,2478067.0,1169773.0,1.553383e+06,1627246.0,3.970320e+05
1,Train_00001,2,1,1,2,1,1,1,36098,36098,36098.000000,36098.0,0.000000,1821027.0,289554.0,1.055290e+06,1055290.5,1.082915e+06
2,Train_00002,2,1,1,3,3,1,3,162423,24980,57329.583333,34541.5,49227.699521,2665430.0,575019.0,1.454428e+06,625491.0,1.069315e+06
3,Train_00003,2,1,1,5,2,1,2,188880,25315,61652.454545,36689.0,62929.509668,2563665.0,830115.0,2.243363e+06,2562619.0,6.976849e+05
4,Train_00004,1,0,1,1,0,0,0,36689,36689,36689.000000,36689.0,0.000000,2014022.0,2014022.0,2.014022e+06,2014022.0,0.000000e+00


In [56]:
trans_df.shape

(41560, 18)

## 数据合并

In [134]:
# train_df = pd.merge(df_base, train_label, on='user', how='left')
train_df = df_base
# train_df = pd.merge(df_base, op_df, on='user', how='left')
# train_df = pd.merge(train_df, trans_df, on='user', how='left')

In [135]:
train=train_df

In [136]:
train.head()

,user,regist_type,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,label,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period_avg,login_cnt_period1_avg,login_cnt_period2_avg,using_time_avg,product7_success_cnt,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,sex_category 0,sex_category 1,provider_category 0,provider_category 1,provider_category 2,level_category 0,level_category 1,level_category 2,verified_category 0,verified_category 1,agreement1_category 0,agreement1_category 1,agreement2_category 0,agreement2_category 1,agreement3_category 0,agreement3_category 1,agreement4_category 0,agreement4_category 1
0,Train_06800,0.273967,0.230296,0.203812,0.286672,0.348703,0.281810,0.297835,0.269883,0.332909,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.0,0.057692,0.225496,0.003810,0.003509,0.004135,0.002801,0.883054,0.000000,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,1,0
1,Train_23487,0.273964,0.197882,0.204529,0.217926,0.226309,0.281810,0.297839,0.213411,0.225405,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,1.0,0.125000,0.235239,0.004431,0.003695,0.004076,0.003265,0.783694,0.857143,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0,1,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1
2,Train_36880,0.273967,0.231331,0.257419,0.230183,0.258607,0.184612,0.204396,0.269890,0.225405,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,1.0,0.125000,0.270565,0.014887,0.013557,0.017896,0.008655,0.645769,0.857143,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0
3,Train_35392,0.249028,0.201054,0.185553,0.260362,0.245544,0.281810,0.297848,0.269553,0.197110,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.0,0.057692,0.243241,0.003550,0.002860,0.002696,0.003046,0.775081,0.857143,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0
4,Train_35057,0.310016,0.316468,0.263675,0.205005,0.221704,0.281807,0.297836,0.204594,0.225404,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,1.0,0.120192,0.225519,0.002607,0.001898,0.003001,0.000651,0.811092,0.857143,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1


In [137]:
length = len(train)
train_length = int(0.90*length)

x = train.drop(columns=['user','label'],axis=1)[0:train_length]
y = train['label'].values[0:train_length]
# y = label_smoothing(y)
# 
valid_x = train.drop(columns=['user','label'],axis=1)[train_length:]
valid_y = train['label'].values[train_length:]
# valid_y = label_smoothing(valid_y)

In [138]:
x.shape

(43003, 68)

In [139]:
x = x.fillna(0)
valid_x = valid_x.fillna(0)

In [140]:
x.head()

,regist_type,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period_avg,login_cnt_period1_avg,login_cnt_period2_avg,using_time_avg,product7_success_cnt,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,sex_category 0,sex_category 1,provider_category 0,provider_category 1,provider_category 2,level_category 0,level_category 1,level_category 2,verified_category 0,verified_category 1,agreement1_category 0,agreement1_category 1,agreement2_category 0,agreement2_category 1,agreement3_category 0,agreement3_category 1,agreement4_category 0,agreement4_category 1
0,0.273967,0.230296,0.203812,0.286672,0.348703,0.281810,0.297835,0.269883,0.332909,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.057692,0.225496,0.003810,0.003509,0.004135,0.002801,0.883054,0.000000,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,1,0
1,0.273964,0.197882,0.204529,0.217926,0.226309,0.281810,0.297839,0.213411,0.225405,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,0.125000,0.235239,0.004431,0.003695,0.004076,0.003265,0.783694,0.857143,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0,1,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1
2,0.273967,0.231331,0.257419,0.230183,0.258607,0.184612,0.204396,0.269890,0.225405,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,0.125000,0.270565,0.014887,0.013557,0.017896,0.008655,0.645769,0.857143,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0
3,0.249028,0.201054,0.185553,0.260362,0.245544,0.281810,0.297848,0.269553,0.197110,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.057692,0.243241,0.003550,0.002860,0.002696,0.003046,0.775081,0.857143,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0
4,0.310016,0.316468,0.263675,0.205005,0.221704,0.281807,0.297836,0.204594,0.225404,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,0.120192,0.225519,0.002607,0.001898,0.003001,0.000651,0.811092,0.857143,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1


In [141]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

# model = LogisticRegression()
# model.fit(x,y)

model = LGBMClassifier(n_estimators=1000, 
                       learning_rate=0.045,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       reg_alpha=100,
                       reg_lambda=100)
model.fit(x, y,
          eval_set=(valid_x, valid_y),
          early_stopping_rounds=5
          )

prediction = model.predict_proba(valid_x)[:,1]
auc = roc_auc_score(valid_y,prediction)
auc

[1]	valid_0's binary_logloss: 0.562512
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.561049
[3]	valid_0's binary_logloss: 0.559759
[4]	valid_0's binary_logloss: 0.558798
[5]	valid_0's binary_logloss: 0.55791
[6]	valid_0's binary_logloss: 0.556852
[7]	valid_0's binary_logloss: 0.555976
[8]	valid_0's binary_logloss: 0.555055
[9]	valid_0's binary_logloss: 0.554348
[10]	valid_0's binary_logloss: 0.553841
[11]	valid_0's binary_logloss: 0.553038
[12]	valid_0's binary_logloss: 0.552388
[13]	valid_0's binary_logloss: 0.551634
[14]	valid_0's binary_logloss: 0.550979
[15]	valid_0's binary_logloss: 0.550541
[16]	valid_0's binary_logloss: 0.550128
[17]	valid_0's binary_logloss: 0.549496
[18]	valid_0's binary_logloss: 0.54902
[19]	valid_0's binary_logloss: 0.548517
[20]	valid_0's binary_logloss: 0.548081
[21]	valid_0's binary_logloss: 0.547654
[22]	valid_0's binary_logloss: 0.547156
[23]	valid_0's binary_logloss: 0.54678
[24]	valid_0's binary_logloss: 0

0.6538659591204111

In [ ]:
joblib.dump(model,'model.pkl')

In [71]:
importance_df = pd.DataFrame({
        'column': x.columns,
        'importance': model.feature_importances_,
    }).sort_values(by='importance')[-100:]

In [72]:
importance_df

,column,importance
54,provider_category 2,0
27,service2_cnt,0
47,service_cnt,0
49,service_amt_cnt_avg,0
14,product5_amount,0
13,product4_amount,0
52,provider_category 0,0
53,provider_category 1,0
26,service1_amt,0
25,service1_cnt,0


In [ ]:
# select_feature=importance_df['column']

# x = x[select_feature]
# valid_x = valid_x[select_feature]

In [ ]:
# from catboost import CatBoostClassifier


# model = CatBoostClassifier(
#     iterations=500,
#     random_seed=42,
#     logging_level='Silent'
# )

# model.fit(
#     x.values, y,
#     eval_set=(valid_x.values, valid_y),
# #     logging_level='Verbose',  # you can uncomment this for text output
#     plot=True
# )

# prediction = model.predict_proba(valid_x)[:,1]
# auc = roc_auc_score(valid_y,prediction)
# auc

baseline:
0.6769352059269422

添加trans amount的扩展变量：
0.6782946345739088

添加trans ip3的扩展变量：
0.6791192236191647

添加trans type直接数值化：
0.6796788342378327

添加所有的des特征，并修改了学习率：
0.6823851869318595

添加avg特征：
0.6831416556804824

In [61]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)